In [18]:
import os
import pandas as pd
import numpy as np
import glob
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [19]:
#global path
global_drive_path="/content/drive/MyDrive/ForeCache/interactions"
directory =(str(global_drive_path) +"/stateActionModel/ndsi-2d")

          

In [20]:
fileNames=[]
for filename in os.listdir(directory):
  if filename.startswith('taskname_ndsi-2d-task_') and filename.endswith('gsheet'):
    fileNames.append(filename)

In [11]:
fileNames

['taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.gsheet',
 'taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a.gsheet',
 'taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.gsheet',
 'taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.gsheet',
 'taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2.gsheet']

In [21]:
fileNames_annotated=['taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2',
 'taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45',
 'taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866',
 'taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a',
 'taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714']

In [22]:
def get_min(time_str):
    """Get minutes from time."""
    time_str=time_str.split(' ')
    h, m, s = time_str[1].split(':')
    s=round(float(s))
    seconds= int(h) * 3600 + int(m) * 60 + int(s)
    return round(seconds/60)

In [23]:
def get_sec(time_str):
    """Get seconds from time."""
    h, m, s = time_str.split(':')
    seconds= int(h) * 3600 + int(m) * 60 + int(s)
    return seconds

In [24]:
#final_dataframe = pd.DataFrame(columns=['Index','Time','State','action','reward','visualization','subtask'])
all_dfs=[]
length=0
for filename in fileNames_annotated:
      print("###########################",filename)
      
      index=[]
      user=[]
      # df = pd.read_csv(os.path.join(directory,filename))
      spreadsheet = gc.open(filename) # Open file using its name. Use this if the file is already anywhere in your drive
      sheet =  spreadsheet.get_worksheet(0)  # 0 means the first sheet in the file
      df = pd.DataFrame(sheet.get_all_records())
      df['State_Zoom']=np.zeros(len(df))
      df['high-level-action']=np.zeros(len(df))
      df['LatitudeLongitude']=np.zeros(len(df))
      df['Location']=np.zeros(len(df))
      length=length+len(df)
      start_time=get_min(df['TimeStamp'][0])
      current_state=None
      for i in range(len(df)):
        # df['Location'][i]=df['Most_frequent_state'][i]
        # df['NDSI'][i]=df['NDSI'][i]
        df['LatitudeLongitude'][i]=int(df['Latitude'][i])+int(df['Longitude'][i])
        if df["State"][i]== current_state:
          df["high-level-action"][i]="same"
        else:
          df["high-level-action"][i]="change"
        current_state=df["State"][i]
        if "Answering" in df["State"][i]:
           df["State"][i]='Answering'
        # elif "Foraging" in df["State"][i]:
        #    df["State"][i]='Foraging / Hypothesis Generation'
        # else:
        #    df["State"][i]='Navigation / Hypothesis Gathering'
        df['State_Zoom'][i]=df['State'][i] + str( df['ZoomLevel'][i])
        df["TimeStamp"][i]=get_min(df["TimeStamp"][i])-start_time  #get minute or get seconds
        index.append(i+1)
        user.append(filename[filename.find("userid_")+7: filename.find("(1).csv")])
        if df["Subtask_Zoom"][i]>= 2:
          df["Subtask_Zoom"]=2
     
      df['Index']=index
      df['User']=user
      df = df[df.State != "Inference Feedback"] #dropped all fields with Inference Feedback
      
      all_dfs.append(df)

########################### taskname_ndsi-2d-task_userid_82316e37-1117-4663-84b4-ddb6455c83b2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher

########################### taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45
########################### taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866
########################### taskname_ndsi-2d-task_userid_7d014c75-453b-4188-9e4e-fb086984896a
########################### taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [25]:
new_df = pd.concat(all_dfs)

In [26]:
new_df.head(10)

,UserIndex,Subtask_2,Action,State,TimeStamp,HighLevelStateActionProbab,StateActionProbab,IdleTime,Reward,Subtask_Screenshot_Time,...,Most_frequent_state,Most_frequent_region,Angle,high-level-action,annotated_subtask,State_Zoom,LatitudeLongitude,Location,Index,User
0,0,0.0,start,Foraging,0,0.666667,0.400000,0.000,0,1,...,None,None,0.000000,change,Subtask_1,Foraging0,-64.0,0.0,1,82316e37-1117-4663-84b4-ddb6455c83b
1,1,1.0,zoom,Navigation,1,0.666667,0.400000,28.039,0,1,...,None,None,0.000000,change,Subtask_1,Navigation1,-62.0,0.0,2,82316e37-1117-4663-84b4-ddb6455c83b
2,2,2.0,pan,Navigation,1,0.500000,0.333333,0.952,0,1,...,Texas,South,344.249312,same,Subtask_1,Navigation1,-38.0,0.0,3,82316e37-1117-4663-84b4-ddb6455c83b
3,3,3.0,zoom,Navigation,1,0.600000,0.428571,1.450,0,1,...,Iowa,UpperMidwest,0.000000,same,Subtask_1,Navigation2,-52.0,0.0,4,82316e37-1117-4663-84b4-ddb6455c83b
4,4,4.0,pan,Navigation,1,0.666667,0.375000,1.061,10,1,...,None,None,10.631352,same,Subtask_1,Navigation2,-77.0,0.0,5,82316e37-1117-4663-84b4-ddb6455c83b
5,5,5.0,pan,Navigation,1,0.714286,0.444444,1.166,10,1,...,None,None,6.449110,same,Subtask_2,Navigation2,-77.0,0.0,6,82316e37-1117-4663-84b4-ddb6455c83b
6,6,6.0,zoom,Navigation,1,0.750000,0.400000,0.781,10,1,...,None,None,0.000000,same,Subtask_2,Navigation3,-55.0,0.0,7,82316e37-1117-4663-84b4-ddb6455c83b
7,7,7.0,zoom,Navigation,1,0.777778,0.454545,0.495,10,1,...,None,NorthWest,0.000000,same,Subtask_2,Navigation4,-70.0,0.0,8,82316e37-1117-4663-84b4-ddb6455c83b
8,8,8.0,pan,Navigation,1,0.800000,0.416667,0.931,10,1,...,Washington,NorthWest,10.166683,same,Subtask_2,Navigation4,-65.0,0.0,9,82316e37-1117-4663-84b4-ddb6455c83b
9,9,9.0,zoom,Navigation,1,0.818182,0.461538,0.805,10,1,...,Washington,NorthWest,0.000000,same,Subtask_2,Navigation5,-74.0,0.0,10,82316e37-1117-4663-84b4-ddb6455c83b


In [27]:
new_df.to_excel(os.path.join(global_drive_path,directory,"annoatated_all_user_NDSI-2D.xlsx"), index=False)

In [28]:
new_df.head(5)

,UserIndex,Subtask_2,Action,State,TimeStamp,HighLevelStateActionProbab,StateActionProbab,IdleTime,Reward,Subtask_Screenshot_Time,...,Most_frequent_state,Most_frequent_region,Angle,high-level-action,annotated_subtask,State_Zoom,LatitudeLongitude,Location,Index,User
0,0,0.0,start,Foraging,0,0.666667,0.400000,0.000,0,1,...,None,None,0.000000,change,Subtask_1,Foraging0,-64.0,0.0,1,82316e37-1117-4663-84b4-ddb6455c83b
1,1,1.0,zoom,Navigation,1,0.666667,0.400000,28.039,0,1,...,None,None,0.000000,change,Subtask_1,Navigation1,-62.0,0.0,2,82316e37-1117-4663-84b4-ddb6455c83b
2,2,2.0,pan,Navigation,1,0.500000,0.333333,0.952,0,1,...,Texas,South,344.249312,same,Subtask_1,Navigation1,-38.0,0.0,3,82316e37-1117-4663-84b4-ddb6455c83b
3,3,3.0,zoom,Navigation,1,0.600000,0.428571,1.450,0,1,...,Iowa,UpperMidwest,0.000000,same,Subtask_1,Navigation2,-52.0,0.0,4,82316e37-1117-4663-84b4-ddb6455c83b
4,4,4.0,pan,Navigation,1,0.666667,0.375000,1.061,10,1,...,None,None,10.631352,same,Subtask_1,Navigation2,-77.0,0.0,5,82316e37-1117-4663-84b4-ddb6455c83b
